# Welcome to Chris The Battle of Neighborhoods (Week 2)

## For this week, I will submit a full report consisting of all of the following components:
* ### Introduction: Business Problem

In this assignment, we want to find a good place to open a Filipino restaurant in New York. There are an estimated 85,000 Filipino immigrants that currently live in New York City, with the largest concentration in the borough of Queens. One of the things that people look for when they have been away for so long is the taste of home. Opening a Filipino restaurant will be a good way of doing that, and it will offer a novel experience to those of other cultures.

* ### Data

I will leverage the data (newyork_data.json) that was used in the previous module. To solve the problem, I will use the Foursquare location data to buid New York neighborhood data and their coordinates. Then I will explore, segment, and use KMeans to cluster the neighborhood based on the top 10 venues. Finally, I will analyze the clustering result and propose the best place to open a Filipino restaurant.

* ### Methodology

I will download and explore a dataset. Explore neighborhoods in New York City. Then I focus my exploration to the borough in Queens. Analyze each neighborhood and group them into 10 clusters. FourSquare will be used to locate all venues. Ratings, tips, and likes by users will be counted and added to the dataframe. Data will be sorted based on rankings. Examine each cluster to decide which cluster is the best place to open a Filipino restaurant. Finally, the data will be visually assessed using Folium.

* ### [Results](#results)

* ### [Discussion](#discussion)

* ### [Conclusion](#conclusion)

Before I get the data and start cleaning it, I'm going to download all the dependencies that I will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

Now I'm going to download and take a look at the data. From this download, I should get a dataset that contains 5 boroughs and 306 neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


I'm going to open and take a quick look at the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

Because all the relevant data is in the features key, which is basically a list of the neighborhoods, I'm going to define a new variable that includes this data and take a look at the first item in this list.

In [4]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Next I'm going to transform this data of nested Python dictionaries into a *pandas* dataframe. To start, I'm going to create an empty dataframe.

In [5]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

I'm going to take a look at the empty dataframe to confirm that the columns are as intended.

In [6]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


I'm going to loop through the data and fill the dataframe one row at a time then examine the resulting dataframe.

In [7]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


I'm going to double-check that the dataset has all 5 boroughs and 306 neighborhoods.

In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


Now I'm going to use geopy library to get the latitude and longitude values of New York City. In order to define an instance of the geocoder, I'm going to need to define a user_agent and I will name the agent *ny_explorer*.

In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Using Folium, I'm going to create a map of New York with neighborhoods superimposed on top.

In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

I'm going simplify the above map by segmenting and clustering only the neighborhoods in Queens. To do that, I'm going to slice the original dataframe and create a new dataframe of the Queens data. 

In [11]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


I'm going to get the geographical coordinates of Queens.

In [12]:
address = 'Queens, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Queens are 40.7498243, -73.7976337.


Now I'm going to visualize Queens and the neighoborhoods in it.

In [13]:
# create map of Queens using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

Next I'm going to start utilizing the Foursquare API to explore the neighborhoods in Queens and segment them. First, I'm going to define my Foursquare Credentials and Version. (Note: Code is hidden for sharing.)

In [40]:
CLIENT_ID = 'Removed for sharing' # your Foursquare ID
CLIENT_SECRET = 'Removed for sharing' # your Foursquare Secret
VERSION = 'Removed for sharing' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Removed for sharing
CLIENT_SECRET:Removed for sharing


Then I'm going to explore the neighborhoods in Queens, get the latitude and longitude values, get the top 100 venues within a radius of 500 meters of each neighborhood.

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now I'm going to write the code to run the above function on each neighborhood and create a new dataframe called queens_venues.

In [16]:
# type your answer here
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


I'm going to check the size of the resulting dataframe.

In [17]:
print(queens_venues.shape)
queens_venues.head()

(2101, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant
3,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
4,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym


I'm going to count how many venues were returned for each neighborhood.

In [18]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,19,19,19,19,19,19
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,19,19,19,19,19,19
Bay Terrace,35,35,35,35,35,35
Bayside,75,75,75,75,75,75
Bayswater,1,1,1,1,1,1
Beechhurst,16,16,16,16,16,16
Bellaire,13,13,13,13,13,13


I'm going to check how many unique categories can be curated from all the returned venues.

In [19]:
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 267 uniques categories.


Now I'm going to analyze each neighborhood.

In [20]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0

I'm going to examine the new dataframe size.

In [21]:
queens_onehot.shape

(2101, 267)

Now I'm going to group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [22]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,Arverne,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

Then I'm going to confirm the new size.

In [23]:
queens_grouped.shape

(81, 267)

I'm going to look at each neighborhood along with the top 5 most common venues.

In [24]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
            venue  freq
0       Surf Spot  0.21
1   Metro Station  0.11
2     Pizza Place  0.05
3  Sandwich Place  0.05
4       BBQ Joint  0.05


----Astoria----
                       venue  freq
0  Middle Eastern Restaurant  0.06
1                        Bar  0.06
2           Greek Restaurant  0.04
3                 Hookah Bar  0.03
4          Indian Restaurant  0.03


----Astoria Heights----
                venue  freq
0  Italian Restaurant  0.15
1         Bus Station  0.08
2        Burger Joint  0.08
3        Gourmet Shop  0.08
4               Plaza  0.08


----Auburndale----
                  venue  freq
0    Italian Restaurant  0.11
1    Athletics & Sports  0.05
2  Fast Food Restaurant  0.05
3      Sushi Restaurant  0.05
4      Toy / Game Store  0.05


----Bay Terrace----
               venue  freq
0     Clothing Store  0.11
1      Women's Store  0.06
2         Kids Store  0.06
3  Mobile Phone Shop  0.06
4     Cosmetics Shop  0.06


----Bayside----
               

Before I put that into a _pandas_ dataframe, I'm going to write a function to sort the venues in descending order.

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then I'm going to create the new dataframe and display the top 10 venues for each neighborhood.

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Pizza Place,BBQ Joint,Sandwich Place,Burrito Place,Thai Restaurant,Coffee Shop,Board Shop,Donut Shop
1,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Indian Restaurant,Hookah Bar,Food Truck,Mediterranean Restaurant,Deli / Bodega,Café,Seafood Restaurant
2,Astoria Heights,Italian Restaurant,Plaza,Bus Station,Playground,Bowling Alley,Supermarket,Gourmet Shop,Bakery,Laundromat,Burger Joint
3,Auburndale,Italian Restaurant,Hookah Bar,Train,Miscellaneous Shop,Mobile Phone Shop,Mattress Store,Fast Food Restaurant,Supermarket,Sushi Restaurant,Noodle House
4,Bay Terrace,Clothing Store,Women's Store,Cosmetics Shop,Mobile Phone Shop,Shoe Store,Donut Shop,Kids Store,American Restaurant,Video Game Store,Pharmacy


Now I'm going to run _k_-means to cluster the neighborhood into 10 clusters.

In [27]:
# set number of clusters
kclusters = 10

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 5], dtype=int32)

I'm going to create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,1,Bar,Middle Eastern Restaurant,Greek Restaurant,Indian Restaurant,Hookah Bar,Food Truck,Mediterranean Restaurant,Deli / Bodega,Café,Seafood Restaurant
1,Queens,Woodside,40.746349,-73.901842,1,Grocery Store,Bakery,Latin American Restaurant,Filipino Restaurant,Thai Restaurant,Pub,American Restaurant,Bar,Donut Shop,Dessert Shop
2,Queens,Jackson Heights,40.751981,-73.882821,1,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Thai Restaurant,Empanada Restaurant,Diner,Pharmacy,Supermarket
3,Queens,Elmhurst,40.744049,-73.881656,1,Mexican Restaurant,Thai Restaurant,Bubble Tea Shop,Vietnamese Restaurant,South American Restaurant,Chinese Restaurant,Park,Colombian Restaurant,Sushi Restaurant,Salon / Barbershop
4,Queens,Howard Beach,40.654225,-73.838138,1,Italian Restaurant,Pharmacy,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Clothing Store,Sushi Restaurant,Breakfast Spot,Japanese Restaurant


Finally, I'm going to visualize the resulting clusters.

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now, I'm going to examine each cluster and determine the discriminating venue categories that distinguish each cluster.

In [30]:
# cluster 1
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,St. Albans,Caribbean Restaurant,Dance Studio,Health Food Store,Fast Food Restaurant,Fried Chicken Joint,Shopping Mall,Café,Event Service,Chinese Restaurant,Donut Shop
39,Cambria Heights,Caribbean Restaurant,Liquor Store,Flower Shop,Cosmetics Shop,Moving Target,Pharmacy,Bakery,Chinese Restaurant,Nightclub,Restaurant
59,Laurelton,Caribbean Restaurant,Cosmetics Shop,Park,Train Station,Farmers Market,Event Service,Event Space,Falafel Restaurant,Farm,Women's Store


In [31]:
# cluster 2
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Bar,Middle Eastern Restaurant,Greek Restaurant,Indian Restaurant,Hookah Bar,Food Truck,Mediterranean Restaurant,Deli / Bodega,Café,Seafood Restaurant
1,Woodside,Grocery Store,Bakery,Latin American Restaurant,Filipino Restaurant,Thai Restaurant,Pub,American Restaurant,Bar,Donut Shop,Dessert Shop
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Thai Restaurant,Empanada Restaurant,Diner,Pharmacy,Supermarket
3,Elmhurst,Mexican Restaurant,Thai Restaurant,Bubble Tea Shop,Vietnamese Restaurant,South American Restaurant,Chinese Restaurant,Park,Colombian Restaurant,Sushi Restaurant,Salon / Barbershop
4,Howard Beach,Italian Restaurant,Pharmacy,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Clothing Store,Sushi Restaurant,Breakfast Spot,Japanese Restaurant
5,Corona,Mexican Restaurant,Chinese Restaurant,Supermarket,Italian Restaurant,Sandwich Place,Latin American Restaurant,South American Restaurant,Restaurant,Bakery,Donut Shop
6,Forest Hills,Gym / Fitness Center,Gym,Pizza Place,Thai Restaurant,Convenience Store,Yoga Studio,Park,Pharmacy,Spa,Supermarket
7,Kew Gardens,Chinese Restaurant,Indian Restaurant,Bank,Cosmetics Shop,Pizza Place,Optical Shop,Park,Bar,Deli / Bodega,Fish Market
8,Richmond Hill,Latin American Restaurant,Bank,Pizza Place,Lounge,Indian Restaurant,Pet Service,Diner,Discount Store,Park,Clothing Store
9,Flushing,Hotpot Restaurant,Bubble Tea Shop,Bakery,Karaoke Bar,Korean Restaurant,Gym / Fitness Center,Food Truck,Gym,Chinese Restaurant,Construction & Landscaping


In [32]:
# cluster 3
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Neponsit,Beach,Women's Store,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food Truck


In [33]:
# cluster 4
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Somerville,Park,Women's Store,Fast Food Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Filipino Restaurant,Electronics Store


In [34]:
# cluster 5
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,Bayswater,Playground,Women's Store,Filipino Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [35]:
# cluster 6
queens_merged.loc[queens_merged['Cluster Labels'] == 5, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Breezy Point,Beach,Monument / Landmark,Supermarket,Trail,Bus Stop,Flea Market,Fish Market,Flower Shop,Fish & Chips Shop,Event Service
49,Rockaway Beach,Beach,Ice Cream Shop,Latin American Restaurant,Deli / Bodega,Bagel Shop,Arepa Restaurant,Seafood Restaurant,BBQ Joint,Brazilian Restaurant,Breakfast Spot
61,Belle Harbor,Beach,Spa,Pub,Deli / Bodega,Chinese Restaurant,Pharmacy,Mexican Restaurant,Italian Restaurant,Restaurant,Boutique
62,Rockaway Park,Beach,Donut Shop,Pizza Place,Mediterranean Restaurant,French Restaurant,Seafood Restaurant,Steakhouse,Gas Station,Board Shop,Sandwich Place
75,Roxbury,Deli / Bodega,Beach,Irish Pub,Trail,Baseball Field,Fast Food Restaurant,Women's Store,Filipino Restaurant,Falafel Restaurant,Farm
78,Hammels,Beach,Bus Station,Building,Fast Food Restaurant,Gym / Fitness Center,Shoe Store,Dog Run,Bus Stop,Fried Chicken Joint,Diner


In [36]:
# cluster 7
queens_merged.loc[queens_merged['Cluster Labels'] == 6, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Brookville,Deli / Bodega,Market,Women's Store,Filipino Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [37]:
# cluster 8
queens_merged.loc[queens_merged['Cluster Labels'] == 7, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Murray Hill,Korean Restaurant,Coffee Shop,Supermarket,Bar,Bank,Automotive Shop,Breakfast Spot,Mobile Phone Shop,Bath House,Karaoke Bar


In [38]:
# cluster 9
queens_merged.loc[queens_merged['Cluster Labels'] == 8, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Floral Park,Indian Restaurant,Basketball Court,Hobby Shop,Grocery Store,Dosa Place,Pizza Place,Wine Shop,Art Gallery,Event Space,Food Stand
54,Jamaica Estates,Indian Restaurant,Bus Station,Empanada Restaurant,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant


In [39]:
# cluster 10
queens_merged.loc[queens_merged['Cluster Labels'] == 9, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,Malba,Tennis Court,Rest Area,Vegetarian / Vegan Restaurant,Empanada Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Women's Store


* ### Results <a name="results"></a>

Looking at the clusters, the first thing that we can see right away is that are a lot of venues offered in cluster 2. Likely because the area covers most of Queens. Cluster 6 has the 2nd most number of venues. While all clusters have a restaurant within 10th Most Common Venues, Filipino restaurants in particular can be found in clusters 2, 3, 4, 5, 6, 7, and 9. In the clusters that have Filipino restaurants, there are also Fish & Chips Shops, Women's Stores, Farmers Markets and other venues that can provide a lot of foot traffic. There are no Filipino restaurants in clusters 1, 8, and 10. 

* ### Discussion <a name="discussion"></a>

With this, the best place to open a Filipino restaurant is in cluster 1 which is located within three neighborhoods: St. Albans, Cambria Heights, and Laurelton. These neighborhoods are in close  proximity of each other. They don't have a Filipino restuarant within the 10th Most Common Venues so we can assume that opening one would be a novel experience which will interest people who want to try out new things. Also, they have the similar venues that appear in other clusters that have Filipino restaurants, which may be helpful in achieving success for the new venue. The next two options would be either clusters 8 and 10 since they don't have Filipino restaurants but with a smaller coverage since they each are only in one neighborhood.

* ### Conclusion <a name="conclusion"></a>

In this report, we took a look at New York City, specifically the borough of Queens, to find the most optimum location to open a Filipino restaurant. We used Foursquare to buid New York neighborhood data and their coordinates, and KMeans to cluster the neighborhood based on the top 10 venues. Based on the results, I recommended that the best place is in cluster 1 due to potential foot traffic and proximity with other neighborhoods. 

Thanks for going through this with me!